<a href="https://colab.research.google.com/github/Yomunozs/Aprendizaje_Maquina/blob/main/Parciales/Parcial_1_TAM_Pr%C3%A1ctica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2 (Valor 2.5 puntos) Genere una simulacion sobre Python de los regresores por m ´ axima verosimilitud y m ´ aximo a-posteriori, discu- ´
tidos en el punto 2.1, para ajustar la senal: ˜ tn = cos[xn/3] + cos[xn/4] + ηn, con xn ∈ [0, 24π], contaminada con ruido blanco
Gaussiano ηn (SNRdB = 2[dB]). Asuma mapeo ϕ(·) del tipo polinomial de orden Q y prior p(w) = N (w|0, σ2
w). Simule 500
datos para entrenar los modelos y 200 para predecir. Incluya normalizacion por MinMaxScaler() de sklearn despu ´ es de generar el ´
mapeo no lineal

Se importan las liberias necesarias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

Simuación de la señal

In [ ]:
N = 700 #Cantidad de datos
x_n = np.linspace(0,24*np.pi,N) #Vector de datos en x
t_n = np.cos(x_n/3)+np.cos(x_n/4) #Vector de datos en y sin ruido

#función cálculo varianza del ruido a partir del snr dB
def var_snr(x,snrdB): #x vector de datos (señal), snrdB SNR en dB
    Px = np.mean(x**2)#estimador potencia media de la señal
    return Px/(10**(snrdB/10)) #Calcuo de varianza del ruido
snrdB = 2 #Relación seal/ruido
var_r = var_snr(t_n,snrdB) #Varianza del ruido a partir de snrDB


t_nr = t_n + np.sqrt(var_r)*np.random.randn(N) #Dtos de la señal contaminada con WGN

#Se convierten los datos en vectores columna
x_n = x_n.reshape(-1,1)#filas = realizaciones-muestras
t_nr = t_nr.reshape(-1,1)


#Grafica de la señal limpia y dispercion de la señal contaminada con WGN
plt.figure(figsize = (5, 3))
plt.plot(x_n, t_n, label='$t_n^*$', c='r', linewidth=2)
plt.scatter(x_n, t_nr, c='b', label='$t_n$')
plt.legend()
plt.xlabel('x_n')
plt.ylabel('t_n')
plt.show()




Mapeo no lineal

In [ ]:
#Division de datos por train_test_split()
#x_train, x_test, t_train, t_test = train_test_split(x_n, t_nr,
#                                                    test_size=200/N, random_state=42)

#print(x_train.shape,x_test.shape,t_train.shape,t_test.shape)

In [ ]:
#generación representación polinomial, desde la libreria sklearn
Q = 12 #grado del polinomio
phiQ = PolynomialFeatures(degree=Q) # generar características polinomiales a partir de los  datos
Phi_xn = phiQ.fit_transform(x_n)#representar datos desde polinomio


In [ ]:
#Division de datos con shuflesplit
rs = ShuffleSplit(n_splits=1, random_state=0, test_size=0.2857) #200 de 700 equivale a  28,57 %
for i, (x_train, x_test) in enumerate(rs.split(Phi_xn)):
  print(i)

#x_train/test, no llevan datos, solo indices



In [ ]:
#Escalamiento de datos
scaler = MinMaxScaler()
Phi_xn[x_train] = scaler.fit_transform(Phi_xn[x_train])
Phi_xn[x_test] = scaler.transform(Phi_xn[x_test])


Modelo de regresion y estimación MCR establecido en sklearn

In [ ]:
#regresor
lambdaR = 1e-15#hiperparámetro de regularización
reg_mc = Ridge(alpha=lambdaR) #Modelo de regresion ridge, L2, alpha ayuda  a evitar sobreajuste

#Se ordenan los indices
x_train = np.sort(x_train)
x_test = np.sort(x_test)

reg_mc.fit(Phi_xn[x_train],t_nr[x_train]) #Se entrena el modelo

t_mc = reg_mc.predict(Phi_xn[x_test])# se prub el modelo con datos nuevos

#Grafica de la señal limpia, dispercion de la señal contaminada con WGN y señal del predict
plt.figure(figsize=(5,3))
plt.plot(x_n,t_n,c='r',label='$t^*$')
plt.scatter(x_n,t_nr,c='b',label='$t$')
plt.plot(x_n[x_test],t_mc,c='g',label='$t_{mcr}$')

plt.legend()
plt.xlabel('x')
plt.ylabel('t')
plt.show()

#raica de los pesos del regresor
plt.figure(figsize=(5,3))
plt.stem(reg_mc.coef_[0])
plt.ylabel('pesos')
plt.show()

In [ ]:
#gráfica de varianza para ML:
var_ml = (np.linalg.norm(t_nr[x_train]-reg_mc.predict(Phi_xn[x_train]))**2)/len(t_nr[x_train]) #Ecuacion de utiizando la predicción del regresor

#Grafica de la variaza ajustada a la señal
plt.figure(figsize=(5,3))
plt.plot(x_n,t_n,c='r',label='$t$')
plt.scatter(x_n,t_nr,c='y',label='$t+\eta$')
plt.plot(x_n[x_test],t_mc,c='g',label='$t_{*}$')
plt.fill_between(x_n[x_test].ravel(), t_mc.ravel() - np.sqrt(var_ml)*np.ones(len(t_mc)),
                t_mc.ravel() + np.sqrt(var_ml)*np.ones(len(t_mc)), alpha=0.8)

plt.legend()
plt.xlabel('x')
plt.ylabel('t')
plt.show()

print(var_ml)

Maxima verosimilitud manual

In [ ]:
#solución Maxima Verosimilitud

#def my_ML(phi, t_n):
#  w_ml = np.linalg.inv(phi.T.dot(phi) + lambdaR*np.eye(phi.shape[1])) @ phi.T @ t_n[x_train]
#
#  s_ml = (np.linalg.norm(t_nr[x_train]-reg_mc.predict(Phi_xn[x_train]))**2)/len(t_nr[x_train])
#
#
#  return w_ml, s_ml


#Solución por svd, valores propios

#Funcionpara calcular los pesos y la varianza
def regsvd(Phi,t,lambdaR=0,tol=1e-16):
  S = Phi.T.dot(Phi) + lambdaR*np.eye(Phi.shape[1]) #Calculo de matriz S, coeficientes
  val,vec = np.linalg.eigh(S) #valores propios de la matriz
  #print(val.shape,vec.shape)
  ind = val > tol #valores propios mayores a 0
  Sinv = vec[:,ind].dot(np.diag(1/val[ind])).dot(vec[:,ind].T)# Se toma solo los valores mayores a cero
  return Sinv.dot(Phi.T.dot(t)),val #Regresa pesos y valores propios

#w, s = my_ML(Phi_xn[x_train], t_nr)
w, s = regsvd(Phi_xn[x_train],t_nr[x_train],lambdaR = lambdaR) #Se calculan los valores de pesos y s con datos de entrenamiento
phiN  = Phi_xn[x_test]#Se toman los datos nuevos que ya tienen la transformacion no lineal
phiN = np.sort(phiN) #Se ordenan los datos por vector de indices
tn =  Phi_xn[x_test] @ w #Se calcula el predict a partir de los pesos y los datos nuevos

#Grafica de señal original contra la señal apartir de svd
plt.figure(figsize=(5,3))
plt.plot(x_n,t_n,c='r',label='$t^*$')
plt.plot(x_n[x_test],tn,c='g',label='$t_{ml}$')

plt.legend()
plt.xlabel('x')
plt.ylabel('t')
plt.show()


In [ ]:


#def my_map():
#return

#...